In [6]:
import os
import sys
import glob
from langchain_community.vectorstores import Chroma
from IPython.display import display, Markdown
from langchain.text_splitter import RecursiveCharacterTextSplitter
import warnings
warnings.filterwarnings('ignore')

current_dir = os.getcwd()
kit_dir = os.path.abspath(os.path.join(current_dir, ".."))
repo_dir = os.path.abspath(os.path.join(kit_dir, ".."))

sys.path.append(kit_dir)
sys.path.append(repo_dir)

from src.routing import Router
from enterprise_knowledge_retriever.src.document_retrieval import DocumentRetrieval

CONFIG_PATH = os.path.join(kit_dir,'config.yaml')

# init router

In [7]:
router = Router(CONFIG_PATH)

# init RAG

In [8]:
folder_loc = os.path.join(kit_dir,'data/')
docx_files = list(glob.glob(f'{folder_loc}/*.txt'))
docs = []
for doc in docx_files:
    with open(doc) as f:
        docs.append(f.read())
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 800,
        chunk_overlap  = 20,
        length_function = len,
        add_start_index = True,
        separators = ["\n\n\n","\n\n", "\n", "*", "."]
    )
text_chunks = text_splitter.create_documents(docs)
document_retrieval =  DocumentRetrieval()
embeddings = document_retrieval.load_embedding_model()
vectorstore = document_retrieval.create_vector_store(text_chunks, embeddings, output_db="data/vectordb/")
document_retrieval.init_retriever(vectorstore)
conversation = document_retrieval.get_qa_retrieval_chain()

2024-09-20 17:59:49,474 [INFO] - Load pretrained SentenceTransformer: intfloat/e5-large-v2


load INSTRUCTOR_Transformer


2024-09-20 17:59:54,436 [INFO] - Use pytorch device: cpu
2024-09-20 17:59:54,438 [INFO] - This is the collection name: collection_2a0f08ef-e1a8-424d-8261-34deff710721


max_seq_length  512
Collection name is None


2024-09-20 17:59:56,578 [INFO] - Vector store saved to data/vectordb/


# user input

In [9]:
query = "How to measure performance using sambatune?"
# query = "Who is the CEO of Sambanova?"
# query = "What is LLM?"

# pipeline

In [10]:
datasource = router.routing(query)
if datasource == "vectorstore":
    output = conversation.invoke({"question":query})
    response = output['answer']
else:
    response = document_retrieval.llm(query)

print(response)
print("--------------------------")
print(f"datasource: {datasource}")

To measure performance using SambaTune, you can run SambaTune with the application YAML file as input. After running `sambatune` and passing in the YAML file, you can then run `sambatune_ui` to view the results. The performance analysis results can be found in the SambaTune GUI, where you can examine reports.

The performance reports are generated after a successful run, which includes collating compile-time and run-time statistics. The reports display in the SambaTune GUI to help identify potential hotspots.

You can also use the `sambatune --help` command to view a list of all options and configuration details.
--------------------------
datasource: vectorstore
